In [3]:
###
#  FUTON Model MDP + Q-Learning Creation Script
#  A Research Project conducted by Noah Dunn 
###

# Import the standard tools for working with Pandas dataframe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shelve
# Import the MDP toolbox that contains a method for conducting Q-Learning
# Tool can be found here: https://github.com/sawcordwell/pymdptoolbox
# Documentation for the tool can be found here 
import mdptoolbox

In [4]:
#  The Data File that will be used to conduct the experiments
patientdata = pd.read_csv("G:/MIMIC-ALL/MIMIC-PATIENTS/patient_data_modified.csv")

In [5]:
### 
#  An MDP, or Markov Decision Process is used to model relationships between various states and actions.
#  A state can be thought of in medical solution as a patient's diagnosis based on current vitals and state of being. 
#  An action can be thought of as a change in current diagnosis based on one of those vitals.
#  The inspirations for the bulk of this code came from Komorowksi's AI Clinician which can be found 
#  here: https://github.com/matthieukomorowski/AI_Clinician/blob/master/AIClinician_core_160219.m
###

###
# Begin by establishing some global variables for use in the MDP creation
###
mdp_count = 500            # The number of repititions we want/count of MDPs we need to create 
clustering_iter = 32       # The number of times clustering will be conducted
cluster_sample = 0.25      # Proportion of the data used for clustering
gamma = 0.99               # How close we desire clusters to be in similarity (Percentage)
transition_threshold = 5   # The cutoff value for the transition matrix
final_policies = 1         # The number of policies we would like to end up with
state_count = 750          # The number of distinct states
action_count = 5           # Number of actions per state (reccommended 2 to 10)
crossval_iter = 10         # Number of crossvalidation runs (Default is 80% Train, 20% Test)

In [6]:
# Python has object serialization to make write/reads fasters, in the form of pickle
import pickle

# Read these values back in from being saved to file
cluster_values = []
cluster_labels = [] 
sample_zscores = []

with open ('cluster_centers.txt', 'rb') as fp:
    cluster_values = pickle.load(fp)
with open ('cluster_labels.txt', 'rb') as fp:
    cluster_labels = pickle.load(fp)
with open ('sample_zscores.txt', 'rb') as fp:
    sample_zscores = pickle.load(fp)
    
print(cluster_values, "\n", "Dimensions: ", len(cluster_values)," x ", len(cluster_values[0]), "\n", sample_zscores)

[[-0.2254902  -0.32352941 -0.42817647 ... -0.07048865  0.20045151
   0.04158715]
 [ 0.09405941 -0.04455446 -0.47672277 ...  0.66545596  0.32843116
   0.42501051]
 [ 0.04545455 -0.15454545 -0.49425455 ...  0.30664516  0.28904419
   0.42852166]
 ...
 [-0.34615385 -0.38461538 -0.46365385 ... -1.47820946 -0.18871191
   0.04611651]
 [ 0.01351351 -0.01351351 -0.46678378 ...  0.34582232  0.109389
   0.14752571]
 [ 0.5        -0.5        -0.5        ... -1.52106461 -2.31206609
  -1.85290276]] 
 Dimensions:  750  x  50 
         gender  mechvent  max_dose_vaso  re_admission  qSOFAFlag  SOFAFlag  \
8         -0.5      -0.5           -0.5     -2.302585        0.5       0.5   
9         -0.5      -0.5           -0.5     -2.302585        0.5       0.5   
10        -0.5      -0.5           -0.5     -2.302585        0.5       0.5   
13         0.5      -0.5           -0.5      0.095310       -0.5      -0.5   
19         0.5      -0.5           -0.5      0.095310        0.5      -0.5   
...        ...

In [7]:
# We now want to use the clusters to determine their nearest real data point neighbors
# As a visual of this. Suppose we have 4 flags of different colors scattered over a park. The K-Means++ algorithm
# is what planted the flags in the middle of groups of people that are similar. The KNN Search (K nearest neighbor search)
# can be used in MatLab as a simple point finder instead of as a more complicated Supervised Learning algorithm. In Python 
# we can make use of the Vector Quanization (vq) package to assign each point to a centroid
from scipy.cluster.vq import vq
closest_clusters = vq(sample_zscores, cluster_values)

# Check to make sure each cluster has a value
print(len(closest_clusters[0]))

# As an aside, closest_clusters[1] contains the distance between each point's values (in this case 50 of them)
# and their closest cluster's values.
# Ex: If a point is [1, 1, 1] and it's closest cluster is the point [3, 3, 3]  closest_clusters[1] would contain the vector
# [abs(3 - 1), abs(3 - 1), abs(3 - 1)] or [2, 2, 2]

# Validate that all the points are in the range 0-749 (since there are only 750 clusters as specified previously)
for i in closest_clusters[0]:
    if(i > 749 or i < 0):
        print("The clusters you are searching for are not configured properly and are out of bounds")
        print("Did you modify the cluster_count variable without changing this error configuration?")
        exit()

47952


In [8]:
### 
#  We want to begin constructing the set of possible actions between states
###

# The number of possible actions is represented as an action_count by action_count matrix
# This is assuming that any action in the list can lead to any other action 
number_actions = action_count * action_count

#  This may prove to be not as useful since this is diagnosis based: extracting information on
#  Fluid input and max dose of vasopressors
iv_fluid = patientdata['input_4hourly']

#  Avoid any fluid that is 0 (That was not administered)
iv_fluid = iv_fluid[iv_fluid > 0]
# Determine minimum and maxium to scale data appropriately
print("Old Lowest IV Fluid Rank: ", min(iv_fluid.rank()))
print("New Highest IV Fluid Rank: ", max(iv_fluid.rank()))
# Now we want to rank these actions in order of their value (lowest to highest)
# We normalize our range from (1.5, 173142.0) to (0, 1)

# Moving the minimum to zero
iv_fluid_ranks = (iv_fluid.rank() - min(iv_fluid.rank()))
# Shifting the max to approximately 1
iv_fluid_ranks = iv_fluid_ranks / max(iv_fluid_ranks)

# Validate that the range is indeed 0 to 1
print("Old Lowest IV Fluid Rank: ", min(iv_fluid_ranks))
print("New Highest IV Fluid Rank: ", max(iv_fluid_ranks))

if round(max(iv_fluid_ranks), 3) != 1 or round(min(iv_fluid_ranks), 3) != 0:
    print("The ranks are not normalized correctly, either the max is too high, or the minium is too low")
    print("Current max: ", round(max(iv_fluid_ranks), 3))
    print("Curret min: ", round(min(iv_fluid_ranks), 3))
    exit()

# This is a mathematics trick to seperate all the values into three distinct groups based on their rank.
# Since ranks are determined based on Vasopressor quantity, the four groups represent the amount of iv fluid
# Administered to a patient (Group 1 - Low, Group 2 - Mid-Low, Group 3 - Mid-High, Group 4 - High)
iv_fluid_groups = np.floor((iv_fluid_ranks + 0.2499999999) * 4)

# Validate that groups are all associated with the numbers 1-4
if not(iv_fluid_groups.isin([1,2,3,4]).any()):
    print("Groups chosen fall outside the desired 1-4 window")
    
# If an IV fluid amount is 0, we denote it to be action 1. 
# If an IV fluid falls into non-zero amounts, we use ranks built above (1 - 4) plus one. Making 
# the subset of these actions to be action 2 thru 5.
# In short, the model can choose to give a 'patient' 5 different IV amounts 
num_of_rows = patientdata['input_4hourly'].size
iv_fluid_actions = pd.Series([1 for i in range(0, num_of_rows)])

# If the value was non-zero and grouped in the 1 - 4 groups, we grab its value to save as an action
for index in iv_fluid_groups.index:
    iv_fluid_actions[index] = iv_fluid_groups[index] + 1

print(iv_fluid_actions)
print(iv_fluid_groups)


Old Lowest IV Fluid Rank:  1.5
New Highest IV Fluid Rank:  173142.0
Old Lowest IV Fluid Rank:  0.0
New Highest IV Fluid Rank:  1.0
0         2
1         3
2         3
3         3
4         2
         ..
238325    1
238326    1
238327    1
238328    1
238329    1
Length: 238330, dtype: int64
0         1.0
1         2.0
2         2.0
3         2.0
4         1.0
         ... 
238319    3.0
238320    4.0
238321    4.0
238322    1.0
238324    2.0
Name: input_4hourly, Length: 173142, dtype: float64


In [97]:
###
# The generate_action_column function takes 4 arguments: 
#
# column_values: A series of column values from a dataframe that we want to turn into action states
# num_groups: How many groups or distinct actions we want to split the data into
# column_name: The name of the column used for print debug statements
# num_rows: The total number of rows in the full column before modifications (This is normally patientdata[column_name].size)
# 
# This function returns column_actions, a series that represents the 'action', or group that each row of data falls under.
#
# An example is found down below, but in words, this function takes a full column of data, groups 
# the values for that data into num_groups distinct actions, and returns a series representing actions based on row
# 
# Ex: Patients' blood pressure might be grouped into 5 categories (Action 1: < 20 mmHg, Action 2: > 20 mmHg && < 60 mmHg... etc)
###

def generate_action_column(column_values, num_groups, column_name, num_rows):
    # Determine minimum and maxium to scale data appropriately
    print("Old Lowest ", column_name, " Rank: ", min(column_values.rank()))
    print("Old Highest " , column_name,  " Rank: ", max(column_values.rank()))
    # Now we want to rank these actions in order of their value (lowest to highest)
    # Normalizing according to lowest and highest rank
    
    # Moving the minimum to zero
    column_ranks = (column_values.rank() - min(column_values.rank()))
    # Shifting the max to approximately 1
    column_ranks = column_ranks / max(column_ranks)

    # Validate that the range is indeed 0 to 1
    print("New Lowest ", column_name, " Rank: ", min(column_ranks))
    print("New Highest ", column_name, " Rank: ", max(column_ranks))

    if round(max(column_ranks), 3) != 1 or round(min(column_ranks), 3) != 0:
        print("The ranks are not normalized correctly, either the max is too high, or the minium is too low")
        print("Current max: ", round(max(column_ranks), 3))
        print("Curret min: ", round(min(column_ranks), 3))
        exit()
    # This is a mathematics trick to seperate all the values into {num_groups} distinct groups based on their rank.
    # Given different columns of interest this can take different forms. For IV fluids, this number is 4.
    column_groups = np.floor(((column_ranks + 1.0/float(num_groups) - 0.000000001) * num_groups))

    # Validate that groups are all associated with desired group split
    if not(iv_fluid_groups.isin([i for i in range(1, num_groups + 1)]).any()):
        print("Groups chosen fall outside the desired 1-4 window")
        exit()
    
    column_actions = pd.Series([1 for i in range(0, num_rows)])

    # If the value was non-zero and grouped in the 1 - 4 groups, we grab its value to save as an action
    for index in column_groups.index:
        column_actions[index] = column_groups[index] + 1

    #print(column_actions)
    #print(column_groups)
    return column_actions
    
    

In [98]:
# This small sample insures the function performs the same as the test conducted above for IV Fluid
iv_fluid = patientdata['input_4hourly']
iv_fluid = iv_fluid[iv_fluid > 0]

test_column = generate_action_column(iv_fluid, 4, "IV Fluid", patientdata['input_4hourly'].size)

print(test_column.equals(iv_fluid_actions))

Old Lowest  IV Fluid  Rank:  1.5
Old Highest  IV Fluid  Rank:  173142.0
New Lowest  IV Fluid  Rank:  0.0
New Highest  IV Fluid  Rank:  1.0
True


In [100]:
# Now we want the exact same thing but done with given Vasopressor amounts
vasopressor_administered = patientdata['max_dose_vaso']
vasopressor_administered = vasopressor_administered[vasopressor_administered > 0]

vasopressor_actions = generate_action_column(vasopressor_administered, 4, "Max Dose Vasopressor", patientdata['max_dose_vaso'].size)
print(vasopressor_actions.unique())

Old Lowest  Max Dose Vasopressor  Rank:  1.0
Old Highest  Max Dose Vasopressor  Rank:  35503.0
New Lowest  Max Dose Vasopressor  Rank:  0.0
New Highest  Max Dose Vasopressor  Rank:  1.0
[1 5 3 4 2]


In [101]:
###
# This function takes two arguments:
# actions_column: A column of action groups generated by the above function (generate_action_column())
# real_values: The actual values from the dataset corresponding to the same column as actions_column
# and returns a list that contains the real median values for each 'group' actions.
#
# Ex: We apply the function to the action_column "IV_Fluid", which has split the data into 4 different groups of 
# IV_Fluid actions. This function will produce a list containing the median amount of IV_Fluid administered for each of those
# groups (Group 1 -> Adminster 20 mL, Group 2 -> Administer 40 mL, Group 3 -> Administer 60 mL, Group 4 -> Administer 80 mL
###

def median_action_values(actions_column, real_values):
    # Grab all the unique actions for a column and sort them
    all_groups = np.sort(actions_column.unique())
    # Concatanate the group number and real value for each row
    action_set = pd.concat([actions_column, real_values], axis=1, sort=False)
    # Name the columns for accurate querying
    action_set.columns = ['group_id', 'data_val']
    # Grab the median value for each group based on group number using python list comprehension
    median_values = [np.median(action_set[action_set['group_id'] == i]['data_val']) for i in all_groups]
    return median_values

    

In [102]:
iv_median_actions = median_action_values(iv_fluid_actions, patientdata['input_4hourly'])
vasopressor_median_actions = median_action_values(vasopressor_actions, patientdata['max_dose_vaso'])
print("IV Action Median Values:", str(iv_median_actions), "\nVasopressor Action Median Values: ", vasopressor_median_actions, "\n")

IV Action Median Values: [0.0, 30.0, 80.66666667, 308.0, 955.5037749999999] 
Vasopressor Action Median Values:  [0.0, 0.04, 0.135, 0.27, 0.7625] 



In [206]:
iv_vaso_groups = pd.concat([iv_fluid_actions, vasopressor_actions], axis=1, sort=False)
iv_vaso_groups.columns = ['iv_group', 'vasopressor_group']

### 
#
### 
def generate_action_matrix(list_action_columns):
    # Drops all group combinations that are duplicates
    list_action_columns = list_action_columns.drop_duplicates(['iv_group', 'vasopressor_group'])
    # Sorts all combinations in order
    list_action_columns = list_action_columns.sort_values(['iv_group', 'vasopressor_group'])
    # Create a list based on the values from the dataframe 
    list_action_columns = list_action_columns.values.tolist()
    print(list_action_columns)

In [207]:
generate_action_matrix(iv_vaso_groups)

[[1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5]]


In [ ]:
# The initial MDP matrix
# We need the values of weights that determines how much the model
# prefers transitioning from one state (medical conditional), to another
# The Matrix must be in the form [[S1][S2][A]] Where S1 is initial state, S2 is the second state, and
# A is the action taken to get from S1 to S2. 
transitions = [[][][]]

# We need to determine the reward value for predicting an outcome leading to survival (+)
# and a penalty for an outcome that will yield death (-)
# The Matrix must be in the form [[S1][S2][R]] Where S1 is initial state, S2 is the second state, and
# R is the reward for taking the action from S1 to S2. 
reward = [[][][]]

# We need to determine the discount value to influence the model to continue changing
# when outcomes are not desired, This value should be kept in the range 0 < discount < 1
discount = 1

# The Q-Learning algorithm will run a fixed number of times
numOfIterations = 10000

# We need to determine whether or not we waant to validate that the transitions and rewards matrix
# to make sure they are valid, this option will only be turned off for speed
scheck = False
